In [43]:
import numpy as np

In [44]:
# Policy parameterization: Softmax
# State value parameterization: Linear
# State feature vector representation: Fourier (as in HW2)

In [45]:
M = 3 # dimensionality of the fourier transform [2, 10]
# mean_policy_vector = np.ones(2*M+1)*0.05 #some non-zero vector

policy_params = np.ones((2*M+1,2))*0.01
value_params = np.ones(2*M+1)*0.01

softmax_sigma = 0.1
# policy_params = np.zeros((np.power(M+1,2),2))
# value_params = np.zeros(np.power(M+1,2))

policy_step_size = 1e-7 #2e-9
valueFunction_step_size = 1e-7
gamma = 1

In [46]:
def phi_s_cosine(given_x, given_v, cosineFlag):
    normal_x = normalized_x(given_x, cosineFlag)
    normal_v = normalized_v(given_v, cosineFlag)
    feature_phi = np.zeros(2*M+1)
#     feature_phi = np.zeros(np.power(M+1,2))
    feature_phi[0] = 1
    if cosineFlag:
        for i in range(1, M+1):
            feature_phi[i] = np.cos(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.cos(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
    
    else:
        for i in range(1, M+1):
            feature_phi[i] = np.sin(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.sin(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))

    return feature_phi

In [47]:
def normalized_x(given_x, cosineFlag):
    if cosineFlag:
        return (given_x+1.2)/(0.5+1.2)
    else:
        return ((given_x+1.2)/(0.5+1.2))*2 - 1
def normalized_v(given_v, cosineFlag):
    if cosineFlag:
        return (given_v+0.7)/(0.7+0.7)
    else:
        return ((given_v+0.7)/(0.7+0.7))*2 - 1

In [48]:
def runEpisode(policy_params, gamma):
    
    episode_trace = []
    
    total_reward = 0
    x_0 = np.random.uniform(-0.6,-0.4,1)[0]
    begin_state = (x_0,0)
    
    S_t = begin_state
    S_t_plus_1 = begin_state
    for t in range(0, 1000):
        
        state_feature_vector = phi_s_cosine(S_t[0], S_t[1], True)
        state_feature_policy_param_product = np.dot(policy_params.T, state_feature_vector)
        exp_terms = np.exp(softmax_sigma * state_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         A_t = np.random.choice([-1,1], p=list(pi_values))
        
        max_action_idx = np.argmax(pi_values)
        if max_action_idx==1:
            A_t = 1
        else:
            A_t = -1
        
#         threshold = np.dot(state_feature_vector, given_theta)
        
#         print(threshold)
        
#         if threshold<=0:
#             A_t = -1
#         else:
#             A_t = 1
        
        v_t_plus_1 = S_t[1] + 0.001*A_t - 0.0025*np.cos(3*S_t[0])
        x_t_plus_1 = S_t[0] + v_t_plus_1
        
        
        
        if x_t_plus_1 < -1.2:
            x_t_plus_1 = -1.2
            v_t_plus_1 = 0
        elif x_t_plus_1 > 0.5:
            x_t_plus_1 = 0.5
            v_t_plus_1 = 0
        
        if v_t_plus_1 < -0.7:
            v_t_plus_1 = -0.7
        elif v_t_plus_1 > 0.7:
            v_t_plus_1 = 0.7
        
        S_t_plus_1 = (x_t_plus_1, v_t_plus_1)
        
        if x_t_plus_1 == 0.5:
            print("Terminated")
            episode_trace.append((S_t, A_t, 0))
            break
        else:
            episode_trace.append((S_t, A_t, -1))
            total_reward += -1
#         print("After ",total_reward)
        

        S_t = S_t_plus_1 #Incrementing time step
    return episode_trace

In [49]:
#With baseline

curr_iter = 0
while(curr_iter<=5000):
    curr_iter += 1
    
    episode_trace = runEpisode(policy_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter)
    policy_params_temp = policy_params.copy()
    for a_step in range(len(episode_trace)):
        G = 0
        for idx in range(a_step, len(episode_trace)):
            G += episode_trace[idx][2]
        
        state_t = episode_trace[a_step][0]
        state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)

        v_hat_s = np.dot(state_t_feature_vector, value_params)
        delta = G - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        

        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
        exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         print("Pi values: ",pi_values)
#         print("G: ",G,". Delta: ",delta)
        if episode_trace[a_step][1] == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    max_diff = np.max(np.abs(policy_params_temp - policy_params))
    print(policy_step_size," Max diff: ",max_diff)
    if max_diff<policy_step_size*1e2 or len(episode_trace)<110:# 0.001 works with 1e-6 policy_step
        break


 EPISODE LENGTH:  1000 CURR ITER:  1
1e-07  Max diff:  0.02512258705838437

 EPISODE LENGTH:  1000 CURR ITER:  2
1e-07  Max diff:  0.02508068100583795

 EPISODE LENGTH:  1000 CURR ITER:  3
1e-07  Max diff:  0.010992233614183798

 EPISODE LENGTH:  1000 CURR ITER:  4
1e-07  Max diff:  0.02006505360894084

 EPISODE LENGTH:  1000 CURR ITER:  5
1e-07  Max diff:  0.025023056545320832

 EPISODE LENGTH:  1000 CURR ITER:  6
1e-07  Max diff:  0.017607793754931187

 EPISODE LENGTH:  1000 CURR ITER:  7
1e-07  Max diff:  0.013125267360710523

 EPISODE LENGTH:  1000 CURR ITER:  8
1e-07  Max diff:  0.012408132648446

 EPISODE LENGTH:  1000 CURR ITER:  9
1e-07  Max diff:  0.022657240145931217

 EPISODE LENGTH:  1000 CURR ITER:  10
1e-07  Max diff:  0.021249566428419304

 EPISODE LENGTH:  1000 CURR ITER:  11
1e-07  Max diff:  0.012732710174992447

 EPISODE LENGTH:  1000 CURR ITER:  12
1e-07  Max diff:  0.010273606275966816

 EPISODE LENGTH:  1000 CURR ITER:  13
1e-07  Max diff:  0.005770231260083344



1e-07  Max diff:  0.01390389020791432

 EPISODE LENGTH:  1000 CURR ITER:  107
1e-07  Max diff:  0.007575065740545655

 EPISODE LENGTH:  1000 CURR ITER:  108
1e-07  Max diff:  0.010044787992191464

 EPISODE LENGTH:  1000 CURR ITER:  109
1e-07  Max diff:  0.015328941157696802

 EPISODE LENGTH:  1000 CURR ITER:  110
1e-07  Max diff:  0.018163689100284163

 EPISODE LENGTH:  1000 CURR ITER:  111
1e-07  Max diff:  0.02428246719038369

 EPISODE LENGTH:  1000 CURR ITER:  112
1e-07  Max diff:  0.02424905727035785

 EPISODE LENGTH:  1000 CURR ITER:  113
1e-07  Max diff:  0.024300545629513302

 EPISODE LENGTH:  1000 CURR ITER:  114
1e-07  Max diff:  0.01440684260123886

 EPISODE LENGTH:  1000 CURR ITER:  115
1e-07  Max diff:  0.011323476895733476

 EPISODE LENGTH:  1000 CURR ITER:  116
1e-07  Max diff:  0.007534911750940725

 EPISODE LENGTH:  1000 CURR ITER:  117
1e-07  Max diff:  0.004820454753120353

 EPISODE LENGTH:  1000 CURR ITER:  118
1e-07  Max diff:  0.006420913989523415

 EPISODE LENGTH:


 EPISODE LENGTH:  1000 CURR ITER:  211
1e-07  Max diff:  0.02366374656795027

 EPISODE LENGTH:  1000 CURR ITER:  212
1e-07  Max diff:  0.02358326173262944

 EPISODE LENGTH:  1000 CURR ITER:  213
1e-07  Max diff:  0.013028499334159498

 EPISODE LENGTH:  1000 CURR ITER:  214
1e-07  Max diff:  0.011019413198723047

 EPISODE LENGTH:  1000 CURR ITER:  215
1e-07  Max diff:  0.014386579143095216

 EPISODE LENGTH:  1000 CURR ITER:  216
1e-07  Max diff:  0.017224857869312535

 EPISODE LENGTH:  1000 CURR ITER:  217
1e-07  Max diff:  0.014279445330414475

 EPISODE LENGTH:  1000 CURR ITER:  218
1e-07  Max diff:  0.013201603963989459

 EPISODE LENGTH:  1000 CURR ITER:  219
1e-07  Max diff:  0.023615747221420118

 EPISODE LENGTH:  1000 CURR ITER:  220
1e-07  Max diff:  0.015098771238217026

 EPISODE LENGTH:  1000 CURR ITER:  221
1e-07  Max diff:  0.008819516044573733

 EPISODE LENGTH:  1000 CURR ITER:  222
1e-07  Max diff:  0.012586324601506185

 EPISODE LENGTH:  1000 CURR ITER:  223
1e-07  Max dif